# Modified 3D PMRNN

## Module Imports

In [1]:
# import numpy as np
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
import os
import cv2

import Utils
# from skimage import util
import pickle

In [2]:
import cv2
print(cv2.__version__)

4.5.3


In [3]:
# from scipy.ndimage import rotate
# # import scipy
# # print(scipy.__version__)

In [4]:
import numpy as np
print(np.__version__)

1.19.5


In [5]:
# rotaxis = [
#     [0, 0, 0],
#     [90, 0, 0],
#     [0, 90, 0],
# ]
# cube = np.ones((10, 10, 10))

# for rot in rotaxis:
#     rotated_cube = cube
#     for i in range(3):
#         rotated_cube = rotate(rotated_cube, angle=rot[i], axes=i, mode='constant', cval=0.0)
#     print(rotated_cube.shape)

In [6]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.5.0


In [7]:
# import pandas as pd
# import matplotlib.pyplot as plt

In [8]:
# import matplotlib
# print(matplotlib.__version__)

In [9]:
from tensorflow.keras import layers,losses,Sequential,metrics
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras import optimizers 
from tensorflow.keras.optimizers import * 
# from tensorflow.keras.optimizers.experimental import SGD
from tensorflow.image import ssim

In [10]:
import sklearn
print(sklearn.__version__)

1.2.2


In [11]:
from CBAM import ConvolutionBlockAttentionModule

In [12]:
from tensorflow.keras import backend as K

In [13]:
import tensorflow._api.v2.compat.v1 as tf1

## Model Config

In [14]:
CONFIGURATION = {
    "BATCH_SIZE": 8,
    "LEARNING_RATE": 0.001,
    "OPTIMIZER":'Adam',
    "N_EPOCHS": 10,
    "N_FILTERS": 6,
    "INPUT_SHAPE":(256,256,1),
    "LATENT_SPACE_DIM" : 64,
    "REDUCED_DIMENSION" : 128,
}

In [15]:
CONFIGURATION["OPTIMIZER"]

'Adam'

## reading data

In [16]:
image_data = Utils.read_data("Berea_2d25um_binary.raw")
image_data.shape

(1000, 1000, 1000)

## Voxel extraction 

In [17]:
voxels = Utils.extract_subvolumes(image_data)
voxels.shape

(64, 250, 256, 256, 1)

In [18]:
del image_data

## Loss Functions

In [19]:
def calculate_kl_loss(model):
    def _calculate_kl_loss(*args):
        kld = tf.keras.losses.KLDivergence()
        kl_loss = kld(model.inferenceDistribution,model.learnedDistribution)
        return tf.abs(kl_loss)
    return _calculate_kl_loss

def _calculate_reconstruction_loss(y_target, y_predicted):
    mse = losses.MeanSquaredError()
    reconstruction_loss = mse( y_target, y_predicted )
    return reconstruction_loss

# def _calculate_reconstruction_loss(y_target, y_predicted):
#     ssim_loss = 1 - tf.reduce_mean(ssim(y_target, y_predicted, max_val=1.0))
#     return ssim_loss


def _calculate_porosity_loss(y_target,y_predicted):
    por1 = tf.reduce_mean(y_target,axis=(1, 2))
    por2 = tf.reduce_mean(y_predicted,axis=(1, 2))
    mae = losses.MeanAbsoluteError()
    return mae(por1,por2)
    

## Model Class

In [20]:
# class EncoderBlock(Model):
#      def __init__(self,n_filter) -> None:
#         super(EncoderBlock,self).__init__()
#         self.conv = layers.Conv2D(n_filter, (3,3) , activation='relu',
#                                padding='same' , name="first_encoder_conv")
#         self.bn = layers.BatchNormalization()
#         self.mp = layers.MaxPooling2D((2,2), padding='same')
            
#     def call(self,X):
#         X = self.conv(X)
#         b = self.bn(X)
#         y = self.mp(b)
#         return y,b

In [21]:
# class DecoderBlock(Model):
#     def __init__(self,n_filter,skip) ->None:
#         super(DecoderBlock,self).__init__()
#         self.convT = layers.Conv2DTranspose(n_filter, (3,3), strides=2, activation='relu'
#                                          , padding='same',name="first_decoder_conv")
#         self.bn = layers.BatchNormalization()
#     def call(self,X):
#         X = self.convT(X)
#         b = self.bn(X)
#         b = layers.Add()([b,skip])
#         return b

In [22]:
# class Encoder(Model):
#     def __init__(self,n_blocks,Filter,latentSpace) -> None:
#         super(Encoder,self).__init__()
#         self.flatten = layers.Flatten()
        
#         self.latentDense = layers.Dense(latentSpace,name ="latent_dense")
#         self.bn = layers.BatchNormalization()
#         self.encBlocks = [EncoderBlock(Filter * (i+1)) for i in range(n_blocks)]
        
#     def call(self,X):
#         for i in range(n_blocks):
#             X = self.encBlocks[i]
#         X = self.flatten(X)
#         X = self.latentDense(X)
#         b = self.bn(X)
#         return b

In [23]:
# 16 * 2 ** (CONFIGURATION["N_FILTERS"]-1)

In [24]:
class Reconstruction():
    """
    Reconstruction represents a Deep Convolutional variational autoencoder architecture
    with mirrored encoder and decoder components.
    """

    def __init__(self,
                 inputShape=CONFIGURATION["INPUT_SHAPE"],
                 latent_space_dim = CONFIGURATION["LATENT_SPACE_DIM"],
                 reducedDimension = CONFIGURATION["REDUCED_DIMENSION"],
                 num_conv_layers = CONFIGURATION["N_FILTERS"],
                 learning_rate = CONFIGURATION["LEARNING_RATE"],
                 batch_size = CONFIGURATION["BATCH_SIZE"],
                 epochs = CONFIGURATION["N_EPOCHS"],
                 opt = CONFIGURATION["OPTIMIZER"],
                )-> None:
        
        
        
        ##### inputs ######
        self.inputShape = inputShape # [256, 256, 1]
        self.latent_space_dim = latent_space_dim # 64
        self.reducedDimension = reducedDimension #256
        self.num_conv_layers = num_conv_layers #5
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.epochs = epochs
        self.opt = opt
#         self.optimizer = getattr(optimizers, opt)(lr = learning_rate) 
        self.optimizers = {
            'SGD' : SGD, 
            'RMSprop' : RMSprop,
            'Adagrad' : Adagrad,
            'Adadelta' : Adadelta,
            'Adam' : Adam,
            'Adamax' : Adamax,
            'Nadam' : Nadam,
        }
        self.reshapeDims = self.inputShape[0] // 2**self.num_conv_layers
        self.last_filter = 16 * 2 ** (self.num_conv_layers-1)
        
        ##### Loss weights ######
        self.reconstruction_loss_weight = 1
        self.Kullback_leibler_weight = 0.001
        self.porosity_Loss_weight = 1
        
        self.skipConnections = None

        self.learnedPrior = None
        self.inference = None
        self.Generate = None
        self.Reconstruction=None

        self.skipConnections = []
        
        self.learnedDistribution = None
        self.inferenceDistribution = None
        
        self._build()

    def summary(self):
        self.learnedPrior.summary()
        self.inference.summary()
        self.Generate.summary()
        self.Reconstruction.summary()
    
    def compile(self):
        optimizer = self.optimizers[self.opt](learning_rate=self.learning_rate)
        self.Reconstruction.compile(optimizer=optimizer,
                           loss=self._calculate_combined_loss,
                           metrics=[_calculate_reconstruction_loss,
                                    _calculate_porosity_loss,
                                    calculate_kl_loss(self)],
                             experimental_run_tf_function=False)
    
    def train(self, inputs1,inputs2):
        return self.Reconstruction.fit(x=[inputs1,inputs2],
                       y=inputs2,
                       batch_size=self.batch_size,
                       epochs=self.epochs,
                       )


    def save(self, save_folder="."):
        self._create_folder_if_it_doesnt_exist(save_folder)
        self._save_parameters(save_folder)
        self._save_weights(save_folder)

    def load_weights(self, weights_path):
        self.Reconstruction.load_weights(weights_path)

    def reconstruct(self, images):
        latent_representations = self.encoder.predict(images)
        reconstructed_images = self.decoder.predict(latent_representations)
        return reconstructed_images, latent_representations
    
    @classmethod
    def load(cls, save_folder="."):
        parameters_path = os.path.join(save_folder, "parameters.pkl")
        with open(parameters_path, "rb") as f:
            parameters = pickle.load(f)
        reconstruction = Reconstruction(*parameters)
        weights_path = os.path.join(save_folder, "weights.h5")
        reconstruction.load_weights(weights_path)
        return reconstruction

    def _create_folder_if_it_doesnt_exist(self, folder):
        if not os.path.exists(folder):
            os.makedirs(folder)

    def _save_parameters(self, save_folder):
        parameters = [
            self.inputShape,
            self.latent_space_dim,
            self.reducedDimension,
            self.num_conv_layers,
            self.learning_rate,
            self.batch_size,
            self.epochs,
            self.opt,
        ]
        save_path = os.path.join(save_folder, "parameters.pkl")
        with open(save_path, "wb") as f:
            pickle.dump(parameters, f)

    def _save_weights(self, save_folder):
        save_path = os.path.join(save_folder, "weights.h5")
        self.Reconstruction.save_weights(save_path)

    def _calculate_combined_loss(self, y_target, y_predicted):
        
        
        reconstruction_loss = _calculate_reconstruction_loss(y_target,y_predicted)
        kl_loss = calculate_kl_loss(self)(self.inferenceDistribution, self.learnedDistribution)
        porisity_loss = _calculate_porosity_loss(y_target, y_predicted)
        combined_loss = self.reconstruction_loss_weight * reconstruction_loss\
                                                         + kl_loss * self.Kullback_leibler_weight\
                                                         + porisity_loss * self.porosity_Loss_weight
        return combined_loss
    


    def _build(self):
        self._build_inference(self.num_conv_layers)
        self._build_learned_prior(self.num_conv_layers)
        self._build_generate(self.num_conv_layers)
        self._build_reconstruction()
    
    
    
    ######  inference ########
        
    def _build_inference(self,num_conv_layers):
        inference_input = layers.Input(shape=self.inputShape, name="inference_input")
        
        ### ------------------------------------ Encoder ---------------------------------------------
        
        x = inference_input
        for i in range(num_conv_layers):
            conv = layers.Conv2D(16 * 2 ** i, (3, 3), activation='relu', padding='same', name=f"encoder_conv_{i+1}")(x)
            bn = layers.BatchNormalization()(conv)
            mp = layers.MaxPooling2D((2, 2), padding='same')(bn)
            x = mp
   
        
        flattened = layers.Flatten()(x)
        
        latentDense = layers.Dense(self.reducedDimension,name ="latent_dense")(flattened)
        conv_out = layers.BatchNormalization()(latentDense)
        
    
        reshaped = layers.Reshape((1, self.reducedDimension))(conv_out)
        inferenceDist = layers.LSTM(self.latent_space_dim, return_sequences=True, name="inference_LSTM")(reshaped)
       
        self.inference = Model(inference_input, inferenceDist, name="inference")
        
        
    ######   learned prior ########
        
    def _build_learned_prior(self,num_conv_layers):
        learned_input = layers.Input(shape=self.inputShape, name="learned_prior_input")
        
        ### ------------------------------------ Encoder ---------------------------------------------
        
        x = learned_input
        for i in range(num_conv_layers):
            conv = layers.Conv2D(16 * 2 ** i, (3, 3), activation='relu', padding='same', name=f"encoder_conv_{i+1}")(x)
            bn = layers.BatchNormalization()(conv)
            mp = layers.MaxPooling2D((2, 2), padding='same')(bn)
            x = mp
        
        flattened = layers.Flatten()(x)
        
        latentDense = layers.Dense(self.reducedDimension,name ="latent_dense")(flattened)
        conv_out = layers.BatchNormalization()(latentDense)
        
        
        
        reshaped = layers.Reshape((1, self.reducedDimension))(conv_out)
        learned_dist = layers.LSTM(self.latent_space_dim, return_sequences=True,name="learned_LSTM")(reshaped)
        self.learnedPrior = Model(learned_input, learned_dist, name="learned_prior")
    
    #### generate ######
    
    def _build_generate(self,num_conv_layers):
        

        
        input_generate = layers.Input(shape=self.inputShape, name="generate_input")
        learnedDist = layers.Input(shape=(1,self.latent_space_dim), name="gen_learned_input")
        
        ### ------------------------------------ Encoder ---------------------------------------------
        
        x = input_generate
        for i in range(num_conv_layers):
            conv = layers.Conv2D(16 * 2 ** i, (3, 3), activation='relu', padding='same', name=f"encoder_conv_{i+1}")(x)
            bn = layers.BatchNormalization()(conv)
            mp = layers.MaxPooling2D((2, 2), padding='same')(bn)
            x = mp
            self.skipConnections.append(bn)

        
        flattened = layers.Flatten()(x)
        
        latentDense = layers.Dense(self.reducedDimension,name ="latent_dense")(flattened)
        conv_out = layers.BatchNormalization()(latentDense)
        
        temp = layers.Reshape(( 1 ,self.reducedDimension ))(conv_out)
        
        concated_input = layers.Concatenate(axis=-1)([temp,learnedDist])
        
        reshaped = layers.Reshape((1, self.reducedDimension + self.latent_space_dim))(concated_input)
        
        generated = layers.LSTM(self.reducedDimension, return_sequences=True,name="generate_LSTM")(reshaped)
        
        ### ------------------------------------ Decoder ---------------------------------------------
        
        upDense = layers.Dense(self.reshapeDims*self.reshapeDims*self.last_filter,name="up_dense")(generated)
        batchNorm_7 = layers.BatchNormalization()(upDense)
        
        reshapedUp = layers.Reshape((self.reshapeDims,self.reshapeDims,self.last_filter))(batchNorm_7)
        
        
        
        generated = reshapedUp
        for i in reversed(range(num_conv_layers)):
            convT = layers.Conv2DTranspose(16 * 2 ** i, (3, 3), strides=2, activation='relu',
                                           padding='same', name=f"decoder_conv_{i+1}")(generated)
            bn = layers.BatchNormalization()(convT)
            skip_conn = self.skipConnections.pop()
            generated = layers.Add()([bn, skip_conn])
#             generated = ConvolutionBlockAttentionModule()(bn)
        
        generated = ConvolutionBlockAttentionModule()(generated)
        gen_out = layers.Conv2DTranspose(1, (3,3), activation='sigmoid', padding='same')(generated)
        
        self.Generate = Model([input_generate,learnedDist], gen_out, name="generate")
        
        
        
    def _build_reconstruction(self):
        input_learned = layers.Input(shape=self.inputShape, name="learned_input")
        input_inference = layers.Input(shape=self.inputShape, name="inference_input")
        

        
        self.learnedDistribution = self.learnedPrior(input_learned)
        self.inferenceDistribution = self.inference(input_inference)
        gen_out = self.Generate([input_learned,self.learnedDistribution])
        
        self.Reconstruction = Model([input_learned,input_inference],gen_out,name="reconstuction")

In [25]:
optimizers = {
            'SGD' : SGD, 
            'RMSprop' : RMSprop,
            'Adagrad' : Adagrad,
            'Adadelta' : Adadelta,
            'Adam' : Adam,
            'Adamax' : Adamax,
            'Nadam' : Nadam,
        }
optimizers['SGD']

tensorflow.python.keras.optimizer_v2.gradient_descent.SGD

## TensorFlow gpu setup

In [26]:
device = tf.config.list_physical_devices("GPU")
print(device)
tf.config.experimental.set_memory_growth(device[0],True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [27]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [28]:
tf.keras.backend.clear_session()

## Model creation

In [29]:
reconstruction = Reconstruction()
reconstruction.summary()
reconstruction.compile()

Instructions for updating:
Colocations handled automatically by placer.
Model: "learned_prior"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
learned_prior_input (InputLa [(None, 256, 256, 1)]     0         
_________________________________________________________________
encoder_conv_1 (Conv2D)      (None, 256, 256, 16)      160       
_________________________________________________________________
batch_normalization_7 (Batch (None, 256, 256, 16)      64        
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 128, 128, 16)      0         
_________________________________________________________________
encoder_conv_2 (Conv2D)      (None, 128, 128, 32)      4640      
_________________________________________________________________
batch_normalization_8 (Batch (None, 128, 128, 32)      128       
_______________________________________________

## train and test data extraction

In [30]:
X_train = voxels[:-6]
X_test = voxels[-6:]

In [31]:
len(X_train)

58

In [32]:
import numpy as np
print(np.__version__)

1.19.5


In [33]:
del voxels

## Creating model input

In [34]:
learnedVoxels = []
inferenceVoxels = []
for voxel in X_train:
    learnedVoxels.append(voxel[:-1])
    inferenceVoxels.append(voxel[1:])

# learnedVoxels = np.array(learnedVoxels)
# inferenceVoxels = np.array(inferenceVoxels)

In [35]:
# import sys
# from mealpy.swarm_based import GWO
# from sklearn.preprocessing import LabelEncoder


# OPT_ENCODER = LabelEncoder()
# OPT_ENCODER.fit(['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'])


# def decode_solution(solution):
#     batch_size = 2**int(solution[0])
    
#     learning_rate = solution[1]
    
#     opt_int = int(solution[2])
#     opt = OPT_ENCODER.inverse_transform([opt_int])[0]
    
#     epoch = 10 *int(solution[3])
    
#     num_filters = int(solution[4])
    
#     latent_space_dim = 2**int(solution[5])
    
#     reduced_dim = 2**int(solution[6])
    
#     return [batch_size,learning_rate,opt,epoch,num_filters,latent_space_dim,reduced_dim]


# def objective_function(solution):
#     batch_size,learning_rate,opt,epoch,num_filters,latent_space_dim ,reduced_dim = decode_solution(solution)
#     reconstruction = Reconstruction(inputShape=CONFIGURATION["INPUT_SHAPE"],
#                                      latent_space_dim = latent_space_dim,
#                                      reducedDimension = reduced_dim,
#                                      num_conv_layers = num_filters,
#                                      learning_rate = learning_rate,
#                                      batch_size = batch_size,
#                                      epochs = epoch,
#                                      opt = opt,
#                                    )
# #     reconstruction.summary()
#     reconstruction.compile()
    
    
#     histories = []
#     for i in range(len(learnedVoxels)):
#         print(f"currently working one voxel : {i +1}, voxels left : {len(learnedVoxels)-(i+1)} ")
#         histories.append(reconstruction.train(learnedVoxels[i],inferenceVoxels[i]))
    
#     reconstruction.save(save_folder=f"model_{batch_size}_{learning_rate}_{opt}_{epoch}_{num_filters}_{latent_space_dim}_{reduced_dim}")
    
#     return histories[-1].history['loss'][-1]
    
    

# LB = [3    , 0.001 , 0   , 2      ,  4  ,  6    , 7 ]
# UB = [5.99 , 0.01  , 6.99, 4.99  ,  8.99,  8.99 , 9.99 ]

# problem_dict = {
#     "fit_func": objective_function,
#     "lb": LB,
#     "ub": UB,
#     "minmax": "min",
#     "verbose":True,
# }

# model = GWO.BaseGWO(problem_dict,epoch=5,pop_size=50)
# model.solve()
# print(f"Best solution : {model.solution[0]}")
# batch_size,learning_rate,opt,epoch,num_filters,latent_space_dim ,reduced_dim = decode_solution(model.solution[0])
# print(f"Batch size : {batch_size}, learning_rate : {learning_rate}, opt : {opt}, epoch : {epoch}, num_filters : {num_filters}, latent_space_dim : {latent_space_dim}, reduced_dim : {reduced_dim}")
# model.history.save_global_objectives_chart(filename="hello/goc")
# model.history.save_local_objectives_chart(filename="hello/loc")

# model.history.save_global_best_fitness_chart(filename="hello/gbfc")
# model.history.save_local_best_fitness_chart(filename="hello/lbfc")

# model.history.save_runtime_chart(filename="hello/rtc")

# model.history.save_exploration_exploitation_chart(filename="hello/eec")

# model.history.save_diversity_chart(filename="hello/dc")

# model.history.save_trajectory_chart(list_agent_idx=[3, 5], selected_dimensions=[3], filename="hello/tc")

## Training the model

In [37]:
histories = []
for i in range(len(learnedVoxels)):
# for i in range(2):
    
        print(f"currently working one voxel : {i +1}, voxels left : {len(learnedVoxels)-(i+1)} ")
        histories.append(reconstruction.train(learnedVoxels[i],inferenceVoxels[i]))

currently working one voxel : 1, voxels left : 57 
Train on 249 samples
Epoch 1/10
249/249 [==============================] - 17s 67ms/sample - loss: 0.3034 - _calculate_reconstruction_loss: 0.1320 - _calculate_porosity_loss: 0.1408 - _calculate_kl_loss: 24.6157
Epoch 2/10
249/249 [==============================] - 10s 39ms/sample - loss: 0.0699 - _calculate_reconstruction_loss: 0.0318 - _calculate_porosity_loss: 0.0149 - _calculate_kl_loss: 22.2113
Epoch 3/10
249/249 [==============================] - 10s 41ms/sample - loss: 0.0543 - _calculate_reconstruction_loss: 0.0286 - _calculate_porosity_loss: 0.0047 - _calculate_kl_loss: 20.4549
Epoch 4/10
249/249 [==============================] - 10s 40ms/sample - loss: 0.0526 - _calculate_reconstruction_loss: 0.0274 - _calculate_porosity_loss: 0.0053 - _calculate_kl_loss: 19.6726
Epoch 5/10
249/249 [==============================] - 10s 42ms/sample - loss: 0.0504 - _calculate_reconstruction_loss: 0.0268 - _calculate_porosity_loss: 0.0045 - _

Epoch 2/10
249/249 [==============================] - 10s 39ms/sample - loss: 0.0407 - _calculate_reconstruction_loss: 0.0236 - _calculate_porosity_loss: 0.0045 - _calculate_kl_loss: 12.4448
Epoch 3/10
249/249 [==============================] - 10s 40ms/sample - loss: 0.0352 - _calculate_reconstruction_loss: 0.0207 - _calculate_porosity_loss: 0.0036 - _calculate_kl_loss: 10.6925
Epoch 4/10
249/249 [==============================] - 10s 39ms/sample - loss: 0.0319 - _calculate_reconstruction_loss: 0.0191 - _calculate_porosity_loss: 0.0035 - _calculate_kl_loss: 9.4906
Epoch 5/10
249/249 [==============================] - 10s 39ms/sample - loss: 0.0301 - _calculate_reconstruction_loss: 0.0178 - _calculate_porosity_loss: 0.0032 - _calculate_kl_loss: 8.8892
Epoch 6/10
249/249 [==============================] - 10s 39ms/sample - loss: 0.0279 - _calculate_reconstruction_loss: 0.0166 - _calculate_porosity_loss: 0.0030 - _calculate_kl_loss: 8.1894
Epoch 7/10
249/249 [============================

249/249 [==============================] - 10s 39ms/sample - loss: 0.0242 - _calculate_reconstruction_loss: 0.0184 - _calculate_porosity_loss: 0.0029 - _calculate_kl_loss: 2.8369
Epoch 4/10
249/249 [==============================] - 10s 40ms/sample - loss: 0.0234 - _calculate_reconstruction_loss: 0.0172 - _calculate_porosity_loss: 0.0034 - _calculate_kl_loss: 2.7367
Epoch 5/10
249/249 [==============================] - 10s 39ms/sample - loss: 0.0213 - _calculate_reconstruction_loss: 0.0159 - _calculate_porosity_loss: 0.0026 - _calculate_kl_loss: 2.6643
Epoch 6/10
249/249 [==============================] - 10s 39ms/sample - loss: 0.0201 - _calculate_reconstruction_loss: 0.0153 - _calculate_porosity_loss: 0.0026 - _calculate_kl_loss: 2.2590
Epoch 7/10
249/249 [==============================] - 10s 39ms/sample - loss: 0.0196 - _calculate_reconstruction_loss: 0.0145 - _calculate_porosity_loss: 0.0031 - _calculate_kl_loss: 1.9678
Epoch 8/10
249/249 [==============================] - 10s 39m

249/249 [==============================] - 10s 39ms/sample - loss: 0.0200 - _calculate_reconstruction_loss: 0.0166 - _calculate_porosity_loss: 0.0026 - _calculate_kl_loss: 0.9555
Epoch 5/10
249/249 [==============================] - 10s 39ms/sample - loss: 0.0195 - _calculate_reconstruction_loss: 0.0158 - _calculate_porosity_loss: 0.0031 - _calculate_kl_loss: 0.9631
Epoch 6/10
249/249 [==============================] - 10s 41ms/sample - loss: 0.0181 - _calculate_reconstruction_loss: 0.0148 - _calculate_porosity_loss: 0.0025 - _calculate_kl_loss: 0.8486
Epoch 7/10
249/249 [==============================] - 10s 40ms/sample - loss: 0.0182 - _calculate_reconstruction_loss: 0.0141 - _calculate_porosity_loss: 0.0030 - _calculate_kl_loss: 1.0031
Epoch 8/10
249/249 [==============================] - 10s 39ms/sample - loss: 0.0169 - _calculate_reconstruction_loss: 0.0134 - _calculate_porosity_loss: 0.0024 - _calculate_kl_loss: 1.1050
Epoch 9/10
249/249 [==============================] - 10s 39m

249/249 [==============================] - 10s 39ms/sample - loss: 0.0182 - _calculate_reconstruction_loss: 0.0147 - _calculate_porosity_loss: 0.0025 - _calculate_kl_loss: 1.0407
Epoch 6/10
249/249 [==============================] - 10s 39ms/sample - loss: 0.0172 - _calculate_reconstruction_loss: 0.0138 - _calculate_porosity_loss: 0.0025 - _calculate_kl_loss: 0.9137
Epoch 7/10
249/249 [==============================] - 10s 39ms/sample - loss: 0.0165 - _calculate_reconstruction_loss: 0.0131 - _calculate_porosity_loss: 0.0024 - _calculate_kl_loss: 1.0232
Epoch 8/10
249/249 [==============================] - 10s 39ms/sample - loss: 0.0164 - _calculate_reconstruction_loss: 0.0127 - _calculate_porosity_loss: 0.0028 - _calculate_kl_loss: 1.0220
Epoch 9/10
249/249 [==============================] - 10s 39ms/sample - loss: 0.0154 - _calculate_reconstruction_loss: 0.0120 - _calculate_porosity_loss: 0.0022 - _calculate_kl_loss: 1.0306
Epoch 10/10
249/249 [==============================] - 10s 39

...

249/249 [==============================] - 10s 38ms/sample - loss: 0.0258 - _calculate_reconstruction_loss: 0.0225 - _calculate_porosity_loss: 0.0025 - _calculate_kl_loss: 0.8806
Epoch 2/10
249/249 [==============================] - 10s 39ms/sample - loss: 0.0208 - _calculate_reconstruction_loss: 0.0178 - _calculate_porosity_loss: 0.0019 - _calculate_kl_loss: 1.0400
Epoch 3/10
249/249 [==============================] - 10s 38ms/sample - loss: 0.0187 - _calculate_reconstruction_loss: 0.0158 - _calculate_porosity_loss: 0.0019 - _calculate_kl_loss: 1.0360
Epoch 4/10
249/249 [==============================] - 10s 38ms/sample - loss: 0.0173 - _calculate_reconstruction_loss: 0.0146 - _calculate_porosity_loss: 0.0017 - _calculate_kl_loss: 1.0357
Epoch 5/10
249/249 [==============================] - 10s 38ms/sample - loss: 0.0169 - _calculate_reconstruction_loss: 0.0137 - _calculate_porosity_loss: 0.0021 - _calculate_kl_loss: 1.1333
Epoch 6/10
249/249 [==============================] - 10s 38m

249/249 [==============================] - 10s 38ms/sample - loss: 0.0231 - _calculate_reconstruction_loss: 0.0201 - _calculate_porosity_loss: 0.0022 - _calculate_kl_loss: 0.8731
Epoch 3/10
249/249 [==============================] - 10s 38ms/sample - loss: 0.0212 - _calculate_reconstruction_loss: 0.0179 - _calculate_porosity_loss: 0.0027 - _calculate_kl_loss: 0.8187
Epoch 4/10
249/249 [==============================] - 10s 38ms/sample - loss: 0.0196 - _calculate_reconstruction_loss: 0.0163 - _calculate_porosity_loss: 0.0024 - _calculate_kl_loss: 0.8424
Epoch 5/10
249/249 [==============================] - 10s 38ms/sample - loss: 0.0185 - _calculate_reconstruction_loss: 0.0153 - _calculate_porosity_loss: 0.0024 - _calculate_kl_loss: 0.8245
Epoch 6/10
249/249 [==============================] - 10s 38ms/sample - loss: 0.0180 - _calculate_reconstruction_loss: 0.0145 - _calculate_porosity_loss: 0.0025 - _calculate_kl_loss: 1.0982
Epoch 7/10
249/249 [==============================] - 10s 38m

249/249 [==============================] - 10s 38ms/sample - loss: 0.0202 - _calculate_reconstruction_loss: 0.0168 - _calculate_porosity_loss: 0.0023 - _calculate_kl_loss: 1.0818
Epoch 4/10
249/249 [==============================] - 10s 38ms/sample - loss: 0.0184 - _calculate_reconstruction_loss: 0.0154 - _calculate_porosity_loss: 0.0021 - _calculate_kl_loss: 0.9097
Epoch 5/10
249/249 [==============================] - 10s 38ms/sample - loss: 0.0177 - _calculate_reconstruction_loss: 0.0145 - _calculate_porosity_loss: 0.0023 - _calculate_kl_loss: 0.9455
Epoch 6/10
249/249 [==============================] - 10s 38ms/sample - loss: 0.0169 - _calculate_reconstruction_loss: 0.0137 - _calculate_porosity_loss: 0.0022 - _calculate_kl_loss: 1.0552
Epoch 7/10
249/249 [==============================] - 10s 38ms/sample - loss: 0.0164 - _calculate_reconstruction_loss: 0.0130 - _calculate_porosity_loss: 0.0023 - _calculate_kl_loss: 1.1056
Epoch 8/10
249/249 [==============================] - 10s 38m

249/249 [==============================] - 10s 38ms/sample - loss: 0.0211 - _calculate_reconstruction_loss: 0.0163 - _calculate_porosity_loss: 0.0034 - _calculate_kl_loss: 1.2831
Epoch 5/10
249/249 [==============================] - 10s 38ms/sample - loss: 0.0195 - _calculate_reconstruction_loss: 0.0149 - _calculate_porosity_loss: 0.0023 - _calculate_kl_loss: 2.3160
Epoch 6/10
249/249 [==============================] - 10s 38ms/sample - loss: 0.0185 - _calculate_reconstruction_loss: 0.0141 - _calculate_porosity_loss: 0.0025 - _calculate_kl_loss: 1.8958
Epoch 7/10
249/249 [==============================] - 10s 38ms/sample - loss: 0.0171 - _calculate_reconstruction_loss: 0.0135 - _calculate_porosity_loss: 0.0023 - _calculate_kl_loss: 1.3068
Epoch 8/10
249/249 [==============================] - 10s 38ms/sample - loss: 0.0166 - _calculate_reconstruction_loss: 0.0130 - _calculate_porosity_loss: 0.0028 - _calculate_kl_loss: 0.7677
Epoch 9/10
249/249 [==============================] - 10s 38m

249/249 [==============================] - 10s 38ms/sample - loss: 0.0182 - _calculate_reconstruction_loss: 0.0149 - _calculate_porosity_loss: 0.0021 - _calculate_kl_loss: 1.2192
Epoch 6/10
249/249 [==============================] - 10s 38ms/sample - loss: 0.0177 - _calculate_reconstruction_loss: 0.0140 - _calculate_porosity_loss: 0.0024 - _calculate_kl_loss: 1.2374
Epoch 7/10
249/249 [==============================] - 10s 38ms/sample - loss: 0.0167 - _calculate_reconstruction_loss: 0.0132 - _calculate_porosity_loss: 0.0022 - _calculate_kl_loss: 1.1665
Epoch 8/10
249/249 [==============================] - 10s 38ms/sample - loss: 0.0164 - _calculate_reconstruction_loss: 0.0127 - _calculate_porosity_loss: 0.0026 - _calculate_kl_loss: 0.9743
Epoch 9/10
249/249 [==============================] - 10s 38ms/sample - loss: 0.0158 - _calculate_reconstruction_loss: 0.0122 - _calculate_porosity_loss: 0.0025 - _calculate_kl_loss: 1.1413
Epoch 10/10
249/249 [==============================] - 10s 38

In [38]:
reconstruction.save(save_folder=f"model_{10}_{0.001}_{'Adam'}_{10}_{6}_{64}_{128}")

In [43]:
saved = Reconstruction.load(save_folder=f"test")

## Testing the model

In [39]:

first  = X_test[0]
Topredict = first[1:]
test = first[:-1]


In [40]:
# predicted = []


# # for i in range(10):
# predicted = reconstruction.Generate.predict([test,Topredict])
# # predicted.append(nextLayer)
# # #     inputLayer = nextLayer
# # predicted.append()

In [41]:
# len(predicted)

In [42]:
# sess = tf.compat.v1.Session()
# predictedValues = []
# for i in range (10):
# # Evaluate the tensor and get its value
#     predictedValues.append(sess.run(predicted[i+1]))


# # Close the session
# sess.close()

In [43]:
# n = 10

# plt.figure(figsize=(20,4))
# for i in range(n):
#     # original
#     ax = plt.subplot(2,n,i+1)
#     plt.imshow(Topredict[i].astype("float32"))
#     plt.title('original')
#     plt.gray()
#     ax.get_xaxis().set_visible(False)
#     ax.get_yaxis().set_visible(False)

#     # reconstructed
#     ax = plt.subplot(2,n,i+1+n)
#     plt.imshow(predicted[i].astype("float32"))
#     plt.title('reconstructed')
#     plt.gray()
#     ax.get_xaxis().set_visible(False)
#     ax.get_yaxis().set_visible(False)

In [44]:
testing = []
inputLayer = first[0]
# for i in range(len(first)-1):
for i in range(10):

    learned = reconstruction.learnedPrior(inputLayer.reshape(1,256,256,1))
    testing.append(reconstruction.Generate.predict([inputLayer.reshape(1,256,256,1),learned],steps=1))
    inputLayer = testing[-1]

D:\Anaconda\envs\PFE\lib\site-packages\tensorflow\python\keras\engine\training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [45]:
len(testing)

10

In [46]:
n = 10

plt.figure(figsize=(20,4))
for i in range(n):
    # original
    ax = plt.subplot(2,n,i+1)
    plt.imshow(Topredict[i].astype("float32"))
    plt.title('original')
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # reconstructed
    ax = plt.subplot(2,n,i+1+n)
    plt.imshow(testing[i].reshape(256,256,1))
    plt.title('reconstructed')
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

NameError: name 'plt' is not defined

In [1]:
# reconstruction.Generate.save('reconstruction_10batch_10epochs.h5')

In [47]:
file_path = "reconstructed.raw"
testing.insert(0, first[0].reshape(1,256,256,1))
print(len(testing))



11


In [48]:
final = np.array(testing,np.float32).reshape(len(testing),256,256)
final.shape

(11, 256, 256)

In [49]:
file_path = "my_array.npy"
np.save("file", final)

In [35]:
final.tofile(file_path)

In [53]:
Volume = np.fromfile(file_path, dtype=np.uint8)
Volume = Volume.reshape(250,256,256)
Volume.shape

(250, 256, 256)